In [1]:
# train with SAC, stable baseline3
import stable_baselines3
from stable_baselines3 import SAC, PPO
from stable_baselines3.sac import MlpPolicy
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.monitor import Monitor
from wandb.integration.sb3 import WandbCallback
import wandb
# pip install gym-robotics
import matplotlib.pyplot as plt
from gym_robotics.envs.fetch.reach import MujocoPyFetchReachEnv
from gym.wrappers import TimeLimit
from jesnk_utils.rgb_to_video import RGB2VIDEO
import cv2
import numpy as np

/research/jesnk_packages/gym_robotics/envs/fetch/reach.py


In [2]:
load_checkpoint_path = "./checkpoint/FetchReach-SAC-dense-20230701_154341.zip"
#load_checkpoint_path = "./checkpoint/PPO-FetchReach-dense-20230621_004058.zip"
model_name = load_checkpoint_path.split("/")[-1].split(".")[0]
rollout_path = f"./checkpoint_rollout/{model_name}/"
model = SAC.load(load_checkpoint_path)

In [3]:
env = MujocoPyFetchReachEnv(reward_type='dense')
env.render_mode = 'rgb_array'
#env = Monitor(env_eval, log_dir)
#env = TimeLimit(env, max_episode_steps=100)
env = DummyVecEnv([lambda: env])
#env.render_mode = 'rgb_array'

episode_step = 0
current_episode = 0
max_episode_num = 1000
cumulative_reward = 0
frames = []
obs = env.reset()
success = []
episodes = []

def init_episode_dict() :
    episode = {}
    # set keys of episode 'observations', 'next_observations', 'actions', 'rewards', 'terminals'
    for key in ['observations', 'next_observations', 'actions', 'rewards', 'terminals']:
        episode[key] = []
    return episode

def convert_ordered_dict_to_array(ordered_dict):
    ret = np.array([])
    for key in ordered_dict.keys():
        ret = np.append(ret, ordered_dict[key])
    return np.array(ret.reshape(-1))

episode = init_episode_dict()

while current_episode < max_episode_num:
    action, _states = model.predict(obs, deterministic=True)
    
    postprocessed_obs = convert_ordered_dict_to_array(obs)
    episode['observations'].append(postprocessed_obs)
    obs, rewards, dones, info = env.step(action)
    cumulative_reward += rewards[0]
    postprocessed_obs = convert_ordered_dict_to_array(obs)
    episode['next_observations'].append(postprocessed_obs)
    episode['actions'].append(action)
    episode['rewards'].append(rewards[0])
    episode['terminals'].append(dones[0])
    
    episode_step += 1
    if dones[0]:
        obs = env.reset()
        success.append(info[0]['is_success'])
        episode_step = 0
        cumulative_reward = 0
        current_episode += 1
        episode['observations'] = np.array(episode['observations'])
        episode['next_observations'] = np.array(episode['next_observations'])
        episode['actions'] = np.array(episode['actions'])
        episode['terminals'] = np.array(episode['terminals'])
        episode['rewards'] = np.array(episode['rewards'])
        episodes.append(episode)
        episode = init_episode_dict()

success_rate = sum(success)/len(success)
print(f'success rate: {success_rate}')

/research/jesnk_packages/gym_robotics/envs/robot_env.py:330: UserWarning: WARN: This version of the mujoco environments depends on the mujoco-py bindings, which are no longer maintained and may stop working. Please upgrade to the v4 versions of the environments (which depend on the mujoco python bindings instead), unless you are trying to precisely replicate previous works).
  logger.warn(
/research/jesnk_packages/gym_robotics/envs/fetch_env.py:256: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  ret = np.array(ret)


success rate: 0.856


In [2]:
from jesnk_utils.utils import convert_ordered_dict_to_array, convert_to_serialized_array
import torch
from gym_robotics.envs.fetch.reach import MujocoPyFetchReachEnv

env = MujocoPyFetchReachEnv(reward_type='dense')
state = convert_to_serialized_array(env.reset())

state_dim = state.shape[0]
device = 'cuda'
cur_state = torch.from_numpy(state).to(device=device).reshape(1, state_dim)


({'observation': array([ 1.34185463e+00,  7.49100507e-01,  5.34707334e-01,  2.00241622e-04,
        6.92290737e-05, -3.14178004e-06, -1.64992873e-09,  5.11115602e-06,
        4.76895682e-06, -2.31822942e-06]), 'achieved_goal': array([1.34185463, 0.74910051, 0.53470733]), 'desired_goal': array([1.35849046, 0.70055753, 0.63110757])}, {})
{'observation': array([ 1.34185463e+00,  7.49100507e-01,  5.34707334e-01,  2.00241622e-04,
        6.92290737e-05, -3.14178004e-06, -1.64992873e-09,  5.11115602e-06,
        4.76895682e-06, -2.31822942e-06]), 'achieved_goal': array([1.34185463, 0.74910051, 0.53470733]), 'desired_goal': array([1.35849046, 0.70055753, 0.63110757])}
{'observation': array([ 1.34185463e+00,  7.49100507e-01,  5.34707334e-01,  2.00241622e-04,
        6.92290737e-05, -3.14178004e-06, -1.64992873e-09,  5.11115602e-06,
        4.76895682e-06, -2.31822942e-06]), 'achieved_goal': array([1.34185463, 0.74910051, 0.53470733]), 'desired_goal': array([1.35849046, 0.70055753, 0.63110757])

In [2]:
import datetime
# get current time from online server
cur_time = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
cur_time

'20230717_043210'

In [5]:
# save data as pickle
import datetime
import pickle
time = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
path = './offline_data/{}'.format(model_name+'.pkl')
print(path)
with open(path, 'wb') as f:
    pickle.dump(episodes, f)
    

./offline_data/FetchReach-SAC-dense-20230701_154341.pkl


In [6]:
env.observation_space

Dict('achieved_goal': Box(-inf, inf, (3,), float64), 'desired_goal': Box(-inf, inf, (3,), float64), 'observation': Box(-inf, inf, (10,), float64))